*TP réalisé par Rachel Blin dans le cadre du cours d'Alexandrina Rogozan*

# Codage de Huffman

L'objectif de ce TP est de réaliser le codage de Huffman pour le message suivant :  
  
"*Il n'existe que deux choses infinies, l'univers et la bêtise humaine... mais pour l'univers, je n'ai pas de certitude absolue.*" (Albert Einstein)

## Simplification de la séquence

Afin de rendre la séquence plus simple à encoder, nous allons tout d'abord transformer la phrase de telle sorte à ce qu'elle ne contienne que des caractères présents dans les 26 lettres de l'alphabet en minuscule, sans accents, ainsi que les espaces.

La message à encoder devient donc :  
  
"*il nexiste que deux choses infinies lunivers et la betise humaine mais pour lunivers je nai pas de certitude absolue*

## Classification des symboles de la séquence par ordre d'occurences croissants

La première étape du codage de Huffman est de classer les symboles de la séquence à encoder par nombre d'occurences croissants.  
  
1) Dans un premier temps, répertoriez tous les caractères présents dans le message.

In [1]:
import numpy as np


m = "il nexiste que deux choses infinies lunivers et la betise humaine mais pour lunivers je nai pas de certitude absolue"


def unique(message):
    """ Returs all the uninque char in a string.
    
    # Argument:
        - message: The string to extract the unique characters from
        
    # Returns:
        - A list containing all the unique characters in the input string.
    """
    chaine = " abcdefghijklmnopqrstuvwxyz"
    rep = list()
    for i in chaine : 
        for j in m : 
            if i == j : 
                rep = rep +  list(i) 
                
    return(np.unique(rep))

unique(m)


array([' ', 'a', 'b', 'c', 'd', 'e', 'f', 'h', 'i', 'j', 'l', 'm', 'n',
       'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x'], dtype='<U1')

2) Calculez maintenant le nombre d'occurences de chacun de ces caractères dans le message et rangez ces occurences par nombre croissant.

In [2]:
import operator
import collections
import pandas as pd

def occurences(message):
    """ Counts the number of occurencies in a given message.
    
    # Argument:
        - message: The string from which we wish to count the occurencies of the chars.
    
    # Returns:
        - A dict containing the chars as keys and the number of occurencies of each chars.
    """
    chaine = " abcdefghijklmnopqrstuvwxyz"
    l = list()
    for i in chaine : 
        for j in message : 
            if i == j : 
                l = l +  list(i) 
                
                
    rep = pd.value_counts(l, ascending = True)
    
    dict(rep)
    
    return(rep)


print(occurences(m))

def order_by_values(dictionary):
    """ Helper function to order a dictionnary by its values.
    
    # Argument:
        - dictionary: The dictionary to order.
        
    # Returns:
        - An ordered list of tuples containing as first value the character and as second the number of occurencies
    
    """
    
    sorted_x = sorted(dictionary.items(), key= lambda couple : ( couple[1], couple[0]))
    return sorted_x



# Nous n'aurons pas besoin de cette fonction car la fonction occurences
# donne déjà un dictionnaire trié par ordre croissant. 
print(order_by_values(occurences(m)))

f     1
q     1
j     1
m     2
c     2
x     2
b     2
p     2
v     2
h     2
o     3
d     3
r     4
l     5
t     5
a     6
n     7
u     8
s    10
i    12
e    17
     19
dtype: int64
[('f', 1), ('j', 1), ('q', 1), ('b', 2), ('c', 2), ('h', 2), ('m', 2), ('p', 2), ('v', 2), ('x', 2), ('d', 3), ('o', 3), ('r', 4), ('l', 5), ('t', 5), ('a', 6), ('n', 7), ('u', 8), ('s', 10), ('i', 12), ('e', 17), (' ', 19)]


In [3]:
x = order_by_values(occurences(m))
x[0][:]

('f', 1)

## Séparation des symboles en deux sous-groupes

Une fois les symboles classés par ordre d'occurence décroissante, on va maintenant créer un arbre dont les feuilles sont les cractères présents dans le message avec leur nombre d'occurences dans le message. Afin de créer cet arbre, on va d'abord chercher les noeuds ayant le plus petit nombre d'occurences et les accrocher à un noeud dont l'occurence est la somme des occurences des deux noeuds. On va répéter cette opération de sorte à ce qu'il n'y ait plus qu'un seul noeud dans l'arbre.

__Exemple__ :  
Pour une séquence contenant les caractères (a, b, c, d, e) d'occurences respectives (1, 1, 2, 2, 3) on obtient le graphe suivant : 

![graphe](TP_Huffman.png)

Ce qui donne donc le codage de Huffman suivant : 

| Caractère | Nombre d'occurences | Code Huffman |
|-----------|---------------------|-------------------|
| a         | 1                   | 111               |
| b         | 1                   | 110               |
| c         | 2                   | 10                |
| d         | 2                   | 01                |
| e         | 3                   | 00                |

3) Créez le graphe de séparation des symboles.

In [11]:
# Création d'une structure de données de type arbre
class Tree(object):
    """ Tree object."""
    def __init__(self, g=None, d=None, data=None):
        """ Init function.
        
        # Arguments:
            - g: The left node of the tree.
            - d: The right node of the tree.
            - data: The data to be held in the node.
        """
        self.g = g
        self.d = d
        self.data = data
        
    def set_data(self, l) :
        self.data = l
        
    def set_fils_g(self, t) : 
        self.g = t
            
    
    def set_fils_d(self, t) : 
        self.d = t 
            
    def rechercher_valeur(self,valeur):
        if self == None : 
                return False
        if valeur in self.data : 
                return True
  
        
        

def update_list_node(list_node, new_node):
    """ Adds a node in the correct place in an ordered list of nodes.
    
    # Arguments:
        - list_node: The list of nodes we want to add the node to.
        - new_node: The node to be added.
    
    # Returns:
        - The list with the added node.
    """

    list_node.append(new_node)
    return sorted(list_node, key= lambda c: c.data[1]) 
            

            
def Huffman(sorted_occurences):
    """ Creates the Huffman tree.
    
    # Argument:
        - sorted_occurences: The sorted occurencies of the message we wish to encode.
    
    # Returns:
        - The root of the Huffman tree.

    """
    list_node = list()
    
    for i in sorted_occurences : 
        update_list_node(list_node,Tree(data = i))
    
    while len(list_node)>2 :
        
        node_g = list_node[0]
        node_d = list_node[1]

        new_node = Tree(data = (node_g.data[0] + node_d.data[0],node_g.data[1] + node_d.data[1]), g = node_g, d= node_d)
        list_node.remove(node_g)
        list_node.remove(node_d)
        list_node = update_list_node(list_node,new_node)

        

    return Tree(data = (list_node[0].data[0] + list_node[1].data[0],list_node[0].data[1] + list_node[1].data[1]), g = list_node[0], d= list_node[1])
    
        


def get_dict_codage_recursif(node,i,a) : 
    if (node == None or not i in node.data[0]) :
        return ''

    else :
        return a + get_dict_codage_recursif(node.g,i,'1') + get_dict_codage_recursif(node.d,i,'0')
 

    
def get_dict_codage(node,sorted_occurences):
    """ Take a coding tree and return the associated coding dictionnary.
    
    # Argument:
        - node: The root node of the coding tree.

    # Return:
        - Dictionnary: The coding dictionnary.

    """
    
    dict_cod = dict(sorted_occurences)
    for i in dict_cod :        
        dict_cod[i] = get_dict_codage_recursif(node.g,i,'1') + get_dict_codage_recursif(node.d,i,'0')

    return dict_cod


In [12]:
len(Huffman(order_by_values(occurences(m))).data[0]) == len(unique(m))

True

In [13]:
get_dict_codage(Huffman(order_by_values(occurences(m))),order_by_values(occurences(m)))

{'f': '110111',
 'j': '110110',
 'q': '100101',
 'b': '100100',
 'c': '011001',
 'h': '011000',
 'm': '001111',
 'p': '001110',
 'v': '001101',
 'x': '001100',
 'd': '11010',
 'o': '10011',
 'r': '01101',
 'l': '1111',
 't': '1110',
 'a': '1100',
 'n': '1000',
 'u': '0111',
 's': '0010',
 'i': '101',
 'e': '010',
 ' ': '000'}

4) Effectuez maintenant le codage de Huffman.

In [19]:
def encode_message(m, dict_Huffman):
    """ Encodes a given message.
    
    # Arguments:
        - m: The message to encode.
        - dict_Huffman: The coding dictionary.
    
    # Returns:
        - The encoded message.
    """
    message_code = ''
    for i in m :
        message_code = message_code + dict_Huffman[i]

    return message_code

In [20]:
m_d = encode_message(m,get_dict_codage(Huffman(order_by_values(occurences(m))),order_by_values(occurences(m))))

## Visualisation de l'encodage 

En se basant sur le codage binaire des caractères de la table ASCII, le message de départ est le suivant :  
"0100100101001100001000000100111001000101010110000100100101010011010101000100010100100000010100010101010101000101001000000100010001000101010101010101100000100000010000110100100001001111010100110100010101010011001000000100100101001110010001100100100101001110010010010100010101010011001000000100110001010101010011100100100101010110010001010101001001010011001000000100010101010100001000000100110001000001001000000100001001000101010101000100100101010011010001010010000001001000010101010100110101000001010010010100111001000101001000000100110101000001010010010101001100100000010100000100111101010101010100100010000001001100010101010100111001001001010101100100010101010010010100110010000001001010010001010010000001001110010000010100100100100000010100000100000101010011001000000100010001000101001000000100001101000101010100100101010001001001010101000101010101000100010001010010000001000001010000100101001101001111010011000101010101000101"    

Pour rappel, le message encodé par la méthode de Shannon-Fano est le suivant :    

"10010011110101101000001100110000100010111000000000111011100100010101100000111000010000011001001100010110001110010101000000011001010110011011000110011011010110010001001010010110001110101000110011010011100010110101000100110001011100001101100011001001100101011011100011001001100110001100011100100101100101110011011010110010001001010010110001100000011011101010100110011100011101001100011001000101110000101010010101000100101000011001000101110100100010110000010010011011101"    

5) Que peut-on dire du rôle du codage de Huffman? Est-il plus ou moins efficace que le codage de Shannon-Fano? Vous calculerez le taux de compression pour illustrer votre réponse.

In [21]:
binaire = "0100100101001100001000000100111001000101010110000100100101010011010101000100010100100000010100010101010101000101001000000100010001000101010101010101100000100000010000110100100001001111010100110100010101010011001000000100100101001110010001100100100101001110010010010100010101010011001000000100110001010101010011100100100101010110010001010101001001010011001000000100010101010100001000000100110001000001001000000100001001000101010101000100100101010011010001010010000001001000010101010100110101000001010010010100111001000101001000000100110101000001010010010101001100100000010100000100111101010101010100100010000001001100010101010100111001001001010101100100010101010010010100110010000001001010010001010010000001001110010000010100100100100000010100000100000101010011001000000100010001000101001000000100001101000101010100100101010001001001010101000101010101000100010001010010000001000001010000100101001101001111010011000101010101000101"

message_Shanon_Fano = "10010011110101101000001100110000100010111000000000111011100100010101100000111000010000011001001100010110001110010101000000011001010110011011000110011011010110010001001010010110001110101000110011010011100010110101000100110001011100001101100011001001100101011011100011001001100110001100011100100101100101110011011010110010001001010010110001100000011011101010100110011100011101001100011001000101110000101010010101000100101000011001000101110100100010110000010010011011101"

def taux_compression(longueur_source, longueur_encode):
    """ Computes the compression rate of the encoded message.
    
    # Arguments:
        - longueur_source: The lenght of the binary message before being encoded.
        - dict_Huffman: The lenght of the binary message after being encoded.
    
    # Returns:
        - The compression rate.
    """   
    return 1-(longueur_encode/longueur_source)
    

In [22]:
print(taux_compression(len(binaire),len(m_d)),taux_compression(len(binaire),len(message_Shanon_Fano)))

0.5043103448275862 0.4967672413793104


C'est le codage de Huffman qui a le meilleur taux de compression ici, 0.504 contre 0.497.

In [23]:
def decoder_message(m, dict_codage_Huffman): 
    
    message_decode = ''
    mot = ''
    
    for i in m :
        mot = mot + i 
        for k,val in dict_codage_Huffman.items():
            if mot == val:
                message_decode = message_decode + k
                mot = ''
    

    return message_decode

decoder_message(m_d,get_dict_codage(Huffman(order_by_values(occurences(m))),order_by_values(occurences(m))) )

'il nexiste que deux choses infinies lunivers et la betise humaine mais pour lunivers je nai pas de certitude absolue'